In [1]:
from time import time
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score, classification_report, confusion_matrix, r2_score, mean_absolute_error, mean_squared_error, median_absolute_error
import sklearn.metrics as metrics
from sklearn.externals import joblib
import numpy as np
from sklearn import svm, datasets
from sklearn.base import clone
import catboost as catb
from catboost import CatBoostClassifier
import xgboost as xgb
from xgboost import XGBClassifier

Using TensorFlow backend.
/Users/Geoffrey/anaconda3/envs/FTDS/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [3]:
y = df_full["SHARPE"]
X = df_full.drop(columns="SHARPE")

In [4]:
from sklearn.preprocessing import Normalizer

scaler = Normalizer()
normalizedX = scaler.fit_transform(X)

In [6]:
def walkforward_model(normalizedX,y,algo):
    recalc_dates = normalizedX.resample('BM',level='date').mean().index.values[:-1]
    
    ## Train models
    models_rolling_window = pd.Series(index=recalc_dates)
    for date in recalc_dates:    
        X_train = normalizedX.xs(slice(date-pd.Timedelta('3652 days'),date),level='date',drop_level=False)
        y_train = y.xs(slice(date-pd.Timedelta('3652 days'),date),level='date',drop_level=False)
        print(f'Train with data prior to: {date} ({y_train.count()} obs)')
        
        model = clone(algo)
        model.fit(X_train,y_train)
        models_rolling_window.loc[date] = model

    begin_dates = models_rolling_window.index
    end_dates = models_rolling_window.index[1:].append(pd.to_datetime(['2099-12-31']))

    ## Generate OUT OF SAMPLE walk-forward predictions
    predictions_rolling_window = pd.Series(index=normalizedX.index)
    for i,model in enumerate(models_rolling_window): #loop thru each models object in collection
        print(f'Using model trained on {begin_dates[i]}, Predict from: {begin_dates[i]} to: {end_dates[i]}')
        X = normalizedX.xs(slice(begin_dates[i],end_dates[i]),level='date',drop_level=False)
        p = pd.Series(model.predict(X),index=X.index)
        predictions_rolling_window.loc[X.index] = p
    
    return models_rolling_window,predictions_rolling_window

In [7]:
XGB_models,XGB_preds = walkforward_model(normalizedX,y,algo=XGBClassifier(*******))

Train with data prior to: 2000-01-31T00:00:00.000000000 (19 obs)
Train with data prior to: 2000-02-29T00:00:00.000000000 (38 obs)
Train with data prior to: 2000-03-31T00:00:00.000000000 (57 obs)
Train with data prior to: 2000-04-28T00:00:00.000000000 (76 obs)
Train with data prior to: 2000-05-31T00:00:00.000000000 (95 obs)
Train with data prior to: 2000-06-30T00:00:00.000000000 (114 obs)
Train with data prior to: 2000-07-31T00:00:00.000000000 (133 obs)
Train with data prior to: 2000-08-31T00:00:00.000000000 (152 obs)
Train with data prior to: 2000-09-29T00:00:00.000000000 (171 obs)
Train with data prior to: 2000-10-31T00:00:00.000000000 (190 obs)
Train with data prior to: 2000-11-30T00:00:00.000000000 (209 obs)
Train with data prior to: 2000-12-29T00:00:00.000000000 (229 obs)
Train with data prior to: 2001-01-31T00:00:00.000000000 (249 obs)
Train with data prior to: 2001-02-28T00:00:00.000000000 (269 obs)
Train with data prior to: 2001-03-30T00:00:00.000000000 (289 obs)
Train with data

Train with data prior to: 2010-05-31T00:00:00.000000000 (2353 obs)
Train with data prior to: 2010-06-30T00:00:00.000000000 (2354 obs)
Train with data prior to: 2010-07-30T00:00:00.000000000 (2355 obs)
Train with data prior to: 2010-08-31T00:00:00.000000000 (2356 obs)
Train with data prior to: 2010-09-30T00:00:00.000000000 (2338 obs)
Train with data prior to: 2010-10-29T00:00:00.000000000 (2358 obs)
Train with data prior to: 2010-11-30T00:00:00.000000000 (2359 obs)
Train with data prior to: 2010-12-31T00:00:00.000000000 (2340 obs)
Train with data prior to: 2011-01-31T00:00:00.000000000 (2360 obs)
Train with data prior to: 2011-02-28T00:00:00.000000000 (2360 obs)
Train with data prior to: 2011-03-31T00:00:00.000000000 (2340 obs)
Train with data prior to: 2011-04-29T00:00:00.000000000 (2360 obs)
Train with data prior to: 2011-05-31T00:00:00.000000000 (2360 obs)
Train with data prior to: 2011-06-30T00:00:00.000000000 (2340 obs)
Train with data prior to: 2011-07-29T00:00:00.000000000 (2360 

Using model trained on 2002-01-31 00:00:00, Predict from: 2002-01-31 00:00:00 to: 2002-02-28 00:00:00
Using model trained on 2002-02-28 00:00:00, Predict from: 2002-02-28 00:00:00 to: 2002-03-29 00:00:00
Using model trained on 2002-03-29 00:00:00, Predict from: 2002-03-29 00:00:00 to: 2002-04-30 00:00:00
Using model trained on 2002-04-30 00:00:00, Predict from: 2002-04-30 00:00:00 to: 2002-05-31 00:00:00
Using model trained on 2002-05-31 00:00:00, Predict from: 2002-05-31 00:00:00 to: 2002-06-28 00:00:00
Using model trained on 2002-06-28 00:00:00, Predict from: 2002-06-28 00:00:00 to: 2002-07-31 00:00:00
Using model trained on 2002-07-31 00:00:00, Predict from: 2002-07-31 00:00:00 to: 2002-08-30 00:00:00
Using model trained on 2002-08-30 00:00:00, Predict from: 2002-08-30 00:00:00 to: 2002-09-30 00:00:00
Using model trained on 2002-09-30 00:00:00, Predict from: 2002-09-30 00:00:00 to: 2002-10-31 00:00:00
Using model trained on 2002-10-31 00:00:00, Predict from: 2002-10-31 00:00:00 to: 

Using model trained on 2009-11-30 00:00:00, Predict from: 2009-11-30 00:00:00 to: 2009-12-31 00:00:00
Using model trained on 2009-12-31 00:00:00, Predict from: 2009-12-31 00:00:00 to: 2010-01-29 00:00:00
Using model trained on 2010-01-29 00:00:00, Predict from: 2010-01-29 00:00:00 to: 2010-02-26 00:00:00
Using model trained on 2010-02-26 00:00:00, Predict from: 2010-02-26 00:00:00 to: 2010-03-31 00:00:00
Using model trained on 2010-03-31 00:00:00, Predict from: 2010-03-31 00:00:00 to: 2010-04-30 00:00:00
Using model trained on 2010-04-30 00:00:00, Predict from: 2010-04-30 00:00:00 to: 2010-05-31 00:00:00
Using model trained on 2010-05-31 00:00:00, Predict from: 2010-05-31 00:00:00 to: 2010-06-30 00:00:00
Using model trained on 2010-06-30 00:00:00, Predict from: 2010-06-30 00:00:00 to: 2010-07-30 00:00:00
Using model trained on 2010-07-30 00:00:00, Predict from: 2010-07-30 00:00:00 to: 2010-08-31 00:00:00
Using model trained on 2010-08-31 00:00:00, Predict from: 2010-08-31 00:00:00 to: 

Using model trained on 2017-11-30 00:00:00, Predict from: 2017-11-30 00:00:00 to: 2017-12-29 00:00:00
Using model trained on 2017-12-29 00:00:00, Predict from: 2017-12-29 00:00:00 to: 2018-01-31 00:00:00
Using model trained on 2018-01-31 00:00:00, Predict from: 2018-01-31 00:00:00 to: 2018-02-28 00:00:00
Using model trained on 2018-02-28 00:00:00, Predict from: 2018-02-28 00:00:00 to: 2018-03-30 00:00:00
Using model trained on 2018-03-30 00:00:00, Predict from: 2018-03-30 00:00:00 to: 2018-04-30 00:00:00
Using model trained on 2018-04-30 00:00:00, Predict from: 2018-04-30 00:00:00 to: 2018-05-31 00:00:00
Using model trained on 2018-05-31 00:00:00, Predict from: 2018-05-31 00:00:00 to: 2018-06-29 00:00:00
Using model trained on 2018-06-29 00:00:00, Predict from: 2018-06-29 00:00:00 to: 2018-07-31 00:00:00
Using model trained on 2018-07-31 00:00:00, Predict from: 2018-07-31 00:00:00 to: 2018-08-31 00:00:00
Using model trained on 2018-08-31 00:00:00, Predict from: 2018-08-31 00:00:00 to: 

In [8]:
filename = 'xgboost_full.pkl'
joblib.dump(XGB_models, filename)

['xgboost_full.pkl']

In [9]:
df_entire = pd.DataFrame(data=XGB_preds).rename(columns={0: "xgb_full_preds"})
df_entire

xgb_full_preds
date       STOCK_TICKER                
2000-01-31 ABT                      0.0
           AGN                      0.0
           ALXN                     2.0
           AMGN                     1.0
           BAX                      0.0
...                                 ...
2019-09-30 LLY                      1.0
           MRK                      1.0
           PFE                      1.0
           REGN                     1.0
           SYK                      1.0

[4627 rows x 1 columns]

In [10]:
df_entire.to_excel("xgboost_full.xlsx", index = True)

In [11]:
def calc_scorecard(y_pred,y_true):
    
    def make_df(y_pred,y_true):
        y_pred.name = 'y_pred'
        y_true.name = 'y_true'

        df = pd.concat([y_pred,y_true],axis=1).dropna()
        
        df['is_correct'] = 0
        df.loc[df.y_pred == df.y_true ,'is_correct'] = 1 # only registers 1 when prediction was made AND it was correct
        df['is_incorrect'] = 0
        df.loc[df.y_pred != df.y_true,'is_incorrect'] = 1 # only registers 1 when prediction was made AND it was wrong
        df['is_correct_0'] = 0
        df.loc[(df.y_pred == df.y_true) & (df.y_pred == 0) ,'is_correct_0'] = 1
        df['is_incorrect_0'] = 0
        df.loc[(df.y_pred != df.y_true) & (df.y_true == 0),'is_incorrect_0'] = 1
        df['is_correct_1'] = 0
        df.loc[(df.y_pred == df.y_true) & (df.y_pred == 1) ,'is_correct_1'] = 1
        df['is_incorrect_1'] = 0
        df.loc[(df.y_pred != df.y_true) & (df.y_true == 1),'is_incorrect_1'] = 1
        df['is_correct_2'] = 0
        df.loc[(df.y_pred == df.y_true) & (df.y_pred == 2) ,'is_correct_2'] = 1
        df['is_incorrect_2'] = 0
        df.loc[(df.y_pred != df.y_true) & (df.y_true == 2),'is_incorrect_2'] = 1
        df['is_predicted'] = df.is_correct + df.is_incorrect
        df['is_predicted_0'] = df.is_correct_0 + df.is_incorrect_0
        df['is_predicted_1'] = df.is_correct_1 + df.is_incorrect_1
        df['is_predicted_2'] = df.is_correct_2 + df.is_incorrect_2
        return df
    
    df = make_df(y_pred,y_true)
    
    scorecard = pd.Series()
    # building block metrics
    scorecard.loc['accuracy'] = df.is_correct.sum()*1. / (df.is_predicted.sum()*1.)*100
    scorecard.loc['accuracy_buy'] = df.is_correct_2.sum()*1. / (df.is_predicted_2.sum()*1.)*100
    scorecard.loc['accuracy_hold'] = df.is_correct_1.sum()*1. / (df.is_predicted_1.sum()*1.)*100
    scorecard.loc['accuracy_sell'] = df.is_correct_0.sum()*1. / (df.is_predicted_0.sum()*1.)*100
    scorecard.loc['RSQ'] = r2_score(df.y_true,df.y_pred)
    scorecard.loc['MAE'] = mean_absolute_error(df.y_true,df.y_pred)
    scorecard.loc['MSE'] = mean_squared_error(df.y_true,df.y_pred)
    scorecard.loc['MedAE'] = median_absolute_error(df.y_true,df.y_pred)
    scorecard.loc['noise'] = df.y_pred.diff().abs().mean()
    # derived metrics
    scorecard.loc['y_true_chg'] = df.y_true.abs().mean()
    scorecard.loc['y_pred_chg'] = df.y_pred.abs().mean()
    
    return scorecard

In [12]:
calc_scorecard(y_pred=XGB_preds,y_true=y).rename('XGBoost')

accuracy         80.981197
accuracy_buy     68.911917
accuracy_hold    89.172802
accuracy_sell    76.724138
RSQ               0.585418
MAE               0.196023
MSE               0.207694
MedAE             0.000000
noise             0.674233
y_true_chg        0.999568
y_pred_chg        0.980333
Name: XGBoost, dtype: float64

In [13]:
def scores_over_time(y_pred,y_true):
    df = pd.concat([y_pred,y_true],axis=1).dropna().reset_index().set_index('date')
    scores = df.resample('BA').apply(lambda df: calc_scorecard(df[y_pred.name],df[y_true.name]))
    return scores

In [14]:
scores_over_time(XGB_preds,y)

,accuracy,accuracy_buy,accuracy_hold,accuracy_sell,RSQ,MAE,MSE,MedAE,noise,y_true_chg,y_pred_chg
date,,,,,,,,,,,
2000-12-29,99.563319,100.000000,99.082569,100.000000,0.991667,0.004367,0.004367,0.0,0.925439,1.000000,1.004367
2001-12-31,96.250000,95.000000,97.500000,95.000000,0.925000,0.037500,0.037500,0.0,0.765690,1.000000,1.012500
2002-12-31,91.363636,83.636364,92.727273,96.363636,0.827273,0.086364,0.086364,0.0,0.835616,1.000000,0.950000
2003-12-31,92.500000,78.333333,95.833333,100.000000,0.850000,0.075000,0.075000,0.0,0.589958,1.000000,0.966667
2004-12-31,92.272727,85.454545,94.545455,94.545455,0.845455,0.077273,0.077273,0.0,0.776256,1.000000,0.986364
2005-12-30,87.083333,93.333333,94.166667,66.666667,0.741667,0.129167,0.129167,0.0,0.799163,1.000000,1.062500
2006-12-29,85.833333,95.000000,91.666667,65.000000,0.716667,0.141667,0.141667,0.0,0.769874,1.000000,1.050000
2007-12-31,81.666667,91.666667,81.666667,71.666667,0.608333,0.187500,0.195833,0.0,0.878661,1.000000,1.029167
2008-12-31,83.333333,76.666667,88.333333,80.000000,0.641667,0.170833,0.179167,0.0,0.757322,1.000000,1.012500


In [15]:
def scores_by_symbol(y_pred,y_true):
    df = pd.concat([y_pred,y_true],axis=1).dropna()
    scores = df.groupby(level='STOCK_TICKER').apply(lambda df: calc_scorecard(df[y_pred.name],df[y_true.name]))
    return scores

In [16]:
scores_by_symbol(XGB_preds,y)

/Users/Geoffrey/anaconda3/envs/FTDS/lib/python3.7/site-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars


,accuracy,accuracy_buy,accuracy_hold,accuracy_sell,RSQ,MAE,MSE,MedAE,noise,y_true_chg,y_pred_chg
STOCK_TICKER,,,,,,,,,,,
ABT,77.155172,7.692308,98.666667,43.478261,0.226083,0.228448,0.228448,0.0,0.108225,0.758621,0.866379
AGN,87.500000,20.000000,98.809524,64.814815,0.478934,0.125000,0.125000,0.0,0.116883,0.810345,0.849138
ALXN,80.000000,85.620915,71.621622,0.000000,0.054559,0.213043,0.239130,0.0,0.078603,1.652174,1.673913
AMGN,86.637931,35.714286,98.958333,8.333333,0.203014,0.133621,0.133621,0.0,0.043290,1.068966,1.047414
BAX,83.620690,33.333333,95.138889,72.857143,0.502258,0.163793,0.163793,0.0,0.125541,0.775862,0.775862
BIIB,79.310345,49.315068,98.648649,18.181818,0.199182,0.215517,0.232759,0.0,0.121212,1.267241,1.155172
BMRN,83.620690,59.259259,98.857143,16.666667,0.332879,0.163793,0.163793,0.0,0.129870,0.987069,1.056034
BMY,85.344828,0.000000,47.272727,98.850575,0.156221,0.159483,0.185345,0.0,0.103896,0.262931,0.120690
CELG,87.931034,99.504950,10.000000,NaN,-0.071947,0.120690,0.120690,0.0,0.021645,1.870690,1.982759
